In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
df1 = pd.read_excel('./data/아모레퍼시픽_시세추이.xlsx')

C:\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# ATR(Average True Range) 변동성 지표 생성

df_atr = df1.iloc[:,[0,1,5,6]]
df_atr.columns = ['date', 'fin', 'high', 'low']

In [4]:
df_atr

,date,fin,high,low
0,2021/08/31,227500,227500,221000
1,2021/08/30,222500,228000,221000
2,2021/08/27,225500,225500,216000
3,2021/08/26,220500,225500,219000
4,2021/08/25,220500,228000,219000
...,...,...,...,...
2460,2011/09/06,1157000,1164000,1126000
2461,2011/09/05,1150000,1172000,1143000
2462,2011/09/02,1159000,1206000,1159000
2463,2011/09/01,1185000,1197000,1151000


In [5]:
lst = []
atr = []

for i in range(0,len(df_atr)-1):
    a = df_atr.iloc[i,2] - df_atr.iloc[i,3]
    b = df_atr.iloc[i,2] - df_atr.iloc[i+1,1]
    c = df_atr.iloc[i,3] - df_atr.iloc[i+1,1]
    lst = [abs(a),abs(b),abs(c)]
    atr.append(max(lst))

In [6]:
# y값(종가 데이터) 준비

fin = list(df_atr[0:-1]['fin'])
del fin[1556:1566] # 거래정지 시점 데이터 행 삭제

In [7]:
# 거래량 데이터 추출

df1 = df1.drop(len(df1)-1,axis=0)
df1 = df1[['일자', '거래량']]

In [8]:
# PER, PBR 추출

df2 = pd.read_excel('./data/아모레퍼시픽_PER.xlsx')
df2 = df2[['일자', 'PER', 'PBR']]

In [9]:
# 거래주체별 순매수 데이터 추출

df3 = pd.read_excel('./data/아모레퍼시픽_거래실적.xlsx')
df3 = df3.drop(columns=['전체'], axis=1)

In [10]:
# 데이터 병합
amore = pd.merge(df1, df2, on='일자', how='outer')
amore = pd.merge(amore, df3, on='일자', how='outer')
amore['ATR'] = atr

In [11]:
# 액면분할에 따른 전처리
amore = amore.drop(index = [1556,1557,1558,1559,1560,1561,1562,1563,1564,1565], axis=1)
amore.iloc[1556:,8] = amore.iloc[1556:,8]/10
amore.iloc[1556:,1] = amore.iloc[1556:,1]*10

In [12]:
# 날짜 데이터로 변환
amore['일자'] = pd.to_datetime(amore['일자'])
amore.rename(columns={'일자':'DATE'}, inplace=True)

In [14]:
# 거시경제 지표와 병합
macro_data = pd.read_csv('./data/merge_data.csv')
macro_data['DATE'] = pd.to_datetime(macro_data['DATE'])    # 날짜 데이터로 변환
macro_data = macro_data.dropna(axis=0)                     # 결측치가 포함된 행 제거
df_amore = pd.merge(amore, macro_data, how='left', on='DATE')

In [15]:
df_amore

,DATE,거래량,PER,PBR,기관 합계,기타법인,개인,외국인 합계,ATR,NASDAQ,S&P,CBOE,Exchange rate,futures2y,futures10y
0,2021-08-31,206720,444.34,3.51,1.992315e+08,-4.684534e+09,-4.683959e+09,9.169262e+09,6500.0,"15,259.20","4,522.68",16.48,"1,158.73",110.32,134.05
1,2021-08-30,149401,434.57,3.44,-1.346435e+09,4.651507e+09,9.890441e+09,-1.319551e+10,7000.0,"15,265.90","4,528.79",16.19,"1,164.34",110.32,134.13
2,2021-08-27,152329,440.43,3.48,7.572624e+09,-2.484023e+09,-3.805818e+09,-1.282782e+09,9500.0,"15,129.50","4,509.37",16.39,"1,161.23",110.30,133.89
3,2021-08-26,116809,430.66,3.41,-6.056925e+09,-6.921380e+08,3.857484e+09,2.891579e+09,6500.0,"14,945.80","4,469.91",18.84,"1,170.53",110.25,133.59
4,2021-08-25,126092,430.66,3.41,-3.491898e+09,6.744800e+08,6.766966e+09,-3.949548e+09,9000.0,"15,041.90","4,496.19",16.79,"1,163.87",110.25,133.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449,2011-09-07,48330,24.60,5.21,9.246510e+08,-1.171000e+07,-5.310770e+08,-3.818640e+08,3300.0,"2,548.90","1,198.62",33.38,"1,069.65",110.27,130.89
2450,2011-09-06,86790,24.31,5.15,1.723145e+09,4.213880e+08,-3.163300e+08,-1.828203e+09,3800.0,"2,473.80","1,165.24",37.00,"1,074.65",110.27,131.22
2451,2011-09-05,73880,24.16,5.12,1.227932e+09,-3.708800e+07,-9.283300e+08,-2.625140e+08,2900.0,NaN,NaN,NaN,NaN,NaN,NaN
2452,2011-09-02,135350,24.35,5.16,-7.265145e+09,-1.335070e+08,-4.972150e+08,7.895867e+09,4700.0,"2,480.30","1,173.97",33.92,"1,065.60",110.25,131.30


In [17]:
# y값(종가 데이터) 병합
df_amore['y'] = fin

In [19]:
# 종가 열 액면분할 반영
df_amore.iloc[1556:,15] = df_amore.iloc[1556:,15]/10

In [20]:
# 결측치를 전일 값으로 대체
df_amore.sort_values('DATE', inplace=True)
df_amore.fillna(method='ffill', inplace=True)

In [25]:
df_amore.to_excel('./data/amore.xlsx',index=False)

In [26]:
amore_df = pd.read_excel('./data/amore.xlsx')

In [27]:
amore_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2454 entries, 0 to 2453
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DATE           2454 non-null   datetime64[ns]
 1   거래량            2454 non-null   int64         
 2   PER            2454 non-null   float64       
 3   PBR            2454 non-null   float64       
 4   기관 합계          2454 non-null   int64         
 5   기타법인           2454 non-null   int64         
 6   개인             2454 non-null   int64         
 7   외국인 합계         2454 non-null   int64         
 8   ATR            2454 non-null   int64         
 9   NASDAQ         2454 non-null   object        
 10  S&P            2454 non-null   object        
 11  CBOE           2454 non-null   float64       
 12  Exchange rate  2454 non-null   object        
 13  futures2y      2454 non-null   float64       
 14  futures10y     2454 non-null   float64       
 15  y              2454 n